Langchain allows you to hook up an LLM to various tools, which gives way to many interesting applications. See https://python.langchain.com/en/latest/modules/agents/tools/getting_started.html#list-of-tools for a list of supported tools.

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os

from dotenv import load_dotenv
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [4]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

## Wikipedia + calculator

In [6]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
prompt = "How much bigger (by what factor) is the population of Amsterdam compared to Utrecht?. Show your calculation."

In [7]:
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to find the population of both cities.
Action: Wikipedia
Action Input: Population of Amsterdam
Observation: Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] (listen); lit. "The Dam on the River Amstel") is the capital and most populous city of the Netherlands, with The Hague being the seat of government. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now designated a UNESCO World Heritage Site.Amsterdam was founded at the mouth of the Amstel River that was dammed to control flooding; the city's name derives from the a local linguistic variation of the word dam. Originally a small fishing village in the late 12th century, Amsterdam became a major world port during the Dutch Go

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


 I now know the final answer.
Final Answer: The population of Amsterdam is 2.5 times bigger than the population of Utrecht.

> Finished chain.


'The population of Amsterdam is 2.5 times bigger than the population of Utrecht.'

## TMDB movie search

In [ ]:
load_dotenv(verbose=True)
TMDB_BEARER_TOKEN = os.environ.get("TMDB_BEARER_TOKEN")

In [ ]:
# TMDB API
tools = load_tools(["tmdb-api"], llm=llm, tmdb_bearer_token=TMDB_BEARER_TOKEN)
prompt = "What is The Imitation Game about?"

In [ ]:
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run(prompt)

### Idea: LLM movie recommendations

Recommend movies to a user based on his preferences

First ask the user a set of questions to get an idea of his preferences. It could go something like this:
```
Q: What genre do you prefer?
    1) action
    2) drama
    3) comedy
    4) don't care
Q: Is there a specific theme you want to explore? Leave blank if you don't care
**user input**
Q: What is the minimum rating the movie should have (0-10)? Leave blank if you don't care
**user input**
Q: Is there a specific actor/actress you want to be in the movie? Leave blank if you don't care
**user input**
```

Then use some sort of movie database (e.g. IMDB) to find the most relevant movies. 

EDIT: after thinking about it, I think this actually be more feasible by using something like the OpenAI embedding API. You could embed, let's say, the movie descriptions of all movies on IMDB with a rating higher than 7.0, and store those embeddings somewhere. Then, you could form a prompt based on the user's preferences (based on the answers provided by the user to the questions above), and simply use e.g. cosine similarity to find suitable matches based on the embeddings.